### Problemas Lista 5

In [1]:
from itertools import repeat, product
from mip import Model, xsum, BINARY, maximize, minimize
import pandas as pd

### Exercicio 1

In [114]:
model = Model()

valores = list(repeat(1, 7)) + list(repeat(0.5, 7)) + list(repeat(0, 7))
coef = dict(zip(range(21), valores))
volume = {0:3.5, 1:3.5, 2:3.5}
garrafas = {0:7, 1:7, 2:7}

x = [[model.add_var(name = "x({}, {})".format(j, t), var_type = BINARY) for t in range(3)] for j in range(21)]
v = [model.add_var(name = "v{}".format(i)) for i in range(3)]

for i in range(3):
    model += xsum([x[j][i]*coef[j] for j in range(21)]) == volume[i] 

for i in range(3):
    model += xsum([x[j][i] for j in range(21)]) == 7


for i in range(21):
    model += xsum([x[i][j] for j in range(3)]) <= 1   


model.optimize()

dados = {}
individuos = list()
volumes = list()
item = []
print('Distribuição de Garrafas: ')
for (i, j) in product(range(21), range(3)):
    if(x[i][j].x > 0.99):
        print(f'Individuo {j}\t volume: {coef[i]}: {x[i][j].x}')
        individuos.append(j)
        volumes.append(coef[i])
        item.append(x[i][j].x)
dados['individuo'] = individuos
dados['item'] = item
dados['volume'] = volumes

df = pd.DataFrame.from_dict(dados)
df = df.sort_values('individuo')
df.groupby(['individuo', 'volume']).sum('item')

Distribuição de Garrafas: 
Individuo 2	 volume: 1: 1.0
Individuo 0	 volume: 1: 1.0
Individuo 0	 volume: 1: 1.0
Individuo 0	 volume: 1: 1.0
Individuo 2	 volume: 1: 1.0
Individuo 1	 volume: 1: 1.0
Individuo 1	 volume: 1: 1.0
Individuo 1	 volume: 0.5: 1.0
Individuo 0	 volume: 0.5: 1.0
Individuo 2	 volume: 0.5: 1.0
Individuo 2	 volume: 0.5: 1.0
Individuo 1	 volume: 0.5: 1.0
Individuo 2	 volume: 0.5: 1.0
Individuo 1	 volume: 0.5: 1.0
Individuo 0	 volume: 0: 1.0
Individuo 1	 volume: 0: 1.0
Individuo 2	 volume: 0: 1.0
Individuo 0	 volume: 0: 1.0
Individuo 1	 volume: 0: 1.0
Individuo 2	 volume: 0: 1.0
Individuo 0	 volume: 0: 1.0


item
individuo volume      
0         0.0      3.0
          0.5      1.0
          1.0      3.0
1         0.0      2.0
          0.5      3.0
          1.0      2.0
2         0.0      2.0
          0.5      3.0
          1.0      2.0

### Exercicio 2

In [248]:
# Definicao das variaveis
Custo = dict(zip(range(1,8), [3.6, 2.3, 4.1, 3.15, 2.8, 2.65, 3.1]))
Pop = dict(zip(range(1,16), [4, 3, 10, 14, 6, 7, 9, 10, 13, 11, 6, 12, 7, 5, 16]))
cobertura = dict(zip(range(1,8), [[1,2], [2,3,5], 
                    [1,7,9,10], [4, 6, 8, 9], [6, 7, 9, 11], [5, 7, 10, 12, 14], [12, 13, 14, 15]]))

# Calculo da populacao atendida por transmissora
n_max = len(Custo) + 1

cobertura_pop = dict()
for j in cobertura.keys():
    p = 0
    for c in cobertura[j]:
        p += Pop[c]
    cobertura_pop[j] = p

# Inicializacao do modelo
model = Model()

x = [model.add_var(name = "x({})".format(i),var_type = BINARY) for i in range(1,9)]

# Funcao Objetivo: Maximizar o atendimento a populacao
model.objective = maximize(xsum(x[i]*cobertura_pop[i] for i in range(1,8)))

# Restricao de investimento
model += xsum(x[i]*Custo[i] for i in range(1,8)) <= 15

model.optimize()

print('Transmissoras: ')
for i in range(1,8):
    if x[i].x >= 0.99:
         print(f"Transmissora {i}")
print('Populacao atendida = {} mil habitantes'.format(model.objective_value))  

model.write('ex_2.lp')

Transmissoras: 
Transmissora 2
Transmissora 4
Transmissora 5
Transmissora 6
Transmissora 7
Populacao atendida = 181.0 mil habitantes


### Exercicio 3

In [4]:
#Exercio 3

# Declaracao dos paramentos
compartimento = [500, 750, 1200, 1500, 1750]
# Considerando a existencia de 4 caminhoes:
cap_transp = [4*x for x in compartimento]

# Demanda por tipo de produto
demandas = {1: 10_000, 2: 15_000, 3: 12_000, 4: 8_000}

# Custo do transporte extra
custo = [0.05, 0.12, 0.08, 0.10]

# Declaracao do modelo
model = Model()

# Variavel represantando o tipo de produto i e o respectivo compartimento do tanque do caminhao
x = [[model.add_var(name = "x({}, {})".format(i,j), var_type = BINARY) for i in range(1,6)] for j in range(1,5)]# 

# Variavel representando a quantidade excedente cujo transporte deverá ser subcontratado
y = [model.add_var(name = "y({})".format(i)) for i in range(1,5)]

# Funcao Objetivo: Minimizar o custo a ser transportado
model.objective = minimize(xsum(y[i]*custo[i] for i in range(4)))

# Restricao de atendimento minimo de demanda por produto
for i in range(4):
    model += xsum(x[i][j]*cap_transp[j] for j in range(5)) + y[i] >= demandas[i+1] 

# Restricao do tipo de produto por compartimento: um compartimento somente poderá transportar um tipo de produto
for j in range(5):
    model += xsum(x[i][j] for i in range(4)) <= 1   
   
model.optimize()

# Custo da subcontratacao
print('Custo = ${} mil'.format(model.objective_value))

# Volume extra a ser contratado para transporte

print('\n')
print('Quantidade a ser transportada por terceiros')
for i in range(4):
    print(f"Produto {i+1}: {y[i].x}")

print('\n')
print('Volume a ser transportado por compartimento')
for (i,j) in product(range(4), range(5)):
    if x[i][j].x > 0.99:
        print(x[i][j].name, x[i][j].x*4*compartimento[j])   

# Salvando o modelo
model.write('ex_4.lp') 

Custo = $1480.0 mil


Quantidade a ser transportada por terceiros
Produto 1: 10000.0
Produto 2: 0.0
Produto 3: 12000.0
Produto 4: 200.0


Volume a ser transportado por compartimento
x(1, 2) 2000.0
x(4, 2) 6000.0
x(5, 2) 7000.0
x(2, 4) 3000.0
x(3, 4) 4800.0
